<a href="https://colab.research.google.com/github/fjadidi2001/Insurance/blob/main/ClaimYN_Prediction_TabNet_Bad_Result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Specify file path
file_path = '/content/drive/My Drive/telematics_syn.csv'

# Import pandas (assuming you want to use it to read the CSV)
import pandas as pd

# Read the CSV file
df = pd.read_csv(file_path)
print(df.shape)  # Should print (100000, 52)
print(df.head()) # To check the first few rows


Mounted at /content/drive
(100000, 52)
   Duration  Insured.age Insured.sex  Car.age  Marital  Car.use  Credit.score  \
0       366           45        Male       -1  Married  Commute         609.0   
1       182           44      Female        3  Married  Commute         575.0   
2       184           48      Female        6  Married  Commute         847.0   
3       183           71        Male        6  Married  Private         842.0   
4       183           84        Male       10  Married  Private         856.0   

  Region  Annual.miles.drive  Years.noclaims  ...  Left.turn.intensity10  \
0  Urban             6213.71              25  ...                    1.0   
1  Urban            12427.42              20  ...                   58.0   
2  Urban            12427.42              14  ...                    0.0   
3  Urban             6213.71              43  ...                    0.0   
4  Urban             6213.71              65  ...                    2.0   

   Left.turn.inte

# pre-process

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Specify file path
file_path = '/content/drive/My Drive/telematics_syn.csv'

# Import pandas (assuming you want to use it to read the CSV)
import pandas as pd

# Read the CSV file
df = pd.read_csv(file_path)
print(df.shape)  # Should print (100000, 52)
print(df.columns) # To check the column names

# Encoding categorical columns using one-hot encoding
categorical_cols = ['Marital', 'Insured.sex', 'Car.use', 'Region', 'Territory']
df_level1 = pd.get_dummies(df, columns=categorical_cols)

# Creating the ClaimYN variable
df_level1['ClaimYN'] = df_level1['NB_Claim'].apply(lambda x: 1 if x > 0 else 0)

# Save the preprocessed data to a new file
preprocessed_file_path_level1 = '/content/drive/My Drive/pre_telematics_syn_level1.csv'
df_level1.to_csv(preprocessed_file_path_level1, index=False)
print(df_level1.shape)  # Should print (100000, number of columns after encoding)

from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Copy the DataFrame from Level 1
df_level2 = df_level1.copy()

# Feature columns (excluding response columns)
feature_cols = [col for col in df_level2.columns if col not in ['NB_Claim', 'AMT_Claim', 'ClaimYN']]

# Applying StandardScaler and MinMaxScaler
scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

df_level2[feature_cols] = scaler.fit_transform(df_level2[feature_cols])
df_level2[feature_cols] = minmax_scaler.fit_transform(df_level2[feature_cols])

# Save the preprocessed data to a new file
preprocessed_file_path_level2 = '/content/drive/My Drive/pre_telematics_syn_level2.csv'
df_level2.to_csv(preprocessed_file_path_level2, index=False)
print(df_level2.shape)  # Should print (100000, number of columns after scaling and normalization)

# Copy the DataFrame from Level 2
df_level3 = df_level2.copy()

# Use provided columns for feature engineering
feature_cols_to_sum = ['Annual.pct.driven', 'Annual.miles.drive', 'Pct.drive.rush am']

# Advanced feature engineering steps
df_level3['DrivingIntensity'] = df_level3[feature_cols_to_sum].sum(axis=1)

# Save the fully preprocessed data to a new file
preprocessed_file_path_level3 = '/content/drive/My Drive/pre_telematics_syn_level3.csv'
df_level3.to_csv(preprocessed_file_path_level3, index=False)
print(df_level3.shape)  # Should print (100000, number of columns after feature engineering)

from sklearn.model_selection import train_test_split

# Define the response and feature columns
response_cols = ['NB_Claim', 'AMT_Claim', 'ClaimYN']


def split_data(df):
    feature_cols = [col for col in df.columns if col not in response_cols]

    # Split into 80% train and 20% test
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

    # Further split train into train and validation
    train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

    return train_df, val_df, test_df, feature_cols

train_df_level1, val_df_level1, test_df_level1, feature_cols_level1 = split_data(df_level1)
train_df_level2, val_df_level2, test_df_level2, feature_cols_level2 = split_data(df_level2)
train_df_level3, val_df_level3, test_df_level3, feature_cols_level3 = split_data(df_level3)

print(train_df_level1.shape, val_df_level1.shape, test_df_level1.shape)
print(train_df_level2.shape, val_df_level2.shape, test_df_level2.shape)
print(train_df_level3.shape, val_df_level3.shape, test_df_level3.shape)




Mounted at /content/drive
(100000, 52)
Index(['Duration', 'Insured.age', 'Insured.sex', 'Car.age', 'Marital',
       'Car.use', 'Credit.score', 'Region', 'Annual.miles.drive',
       'Years.noclaims', 'Territory', 'Annual.pct.driven',
       'Total.miles.driven', 'Pct.drive.mon', 'Pct.drive.tue', 'Pct.drive.wed',
       'Pct.drive.thr', 'Pct.drive.fri', 'Pct.drive.sat', 'Pct.drive.sun',
       'Pct.drive.2hrs', 'Pct.drive.3hrs', 'Pct.drive.4hrs', 'Pct.drive.wkday',
       'Pct.drive.wkend', 'Pct.drive.rush am', 'Pct.drive.rush pm',
       'Avgdays.week', 'Accel.06miles', 'Accel.08miles', 'Accel.09miles',
       'Accel.11miles', 'Accel.12miles', 'Accel.14miles', 'Brake.06miles',
       'Brake.08miles', 'Brake.09miles', 'Brake.11miles', 'Brake.12miles',
       'Brake.14miles', 'Left.turn.intensity08', 'Left.turn.intensity09',
       'Left.turn.intensity10', 'Left.turn.intensity11',
       'Left.turn.intensity12', 'Right.turn.intensity08',
       'Right.turn.intensity09', 'Right.turn.inte

In [ ]:
# Install the library
!pip install pytorch-tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
# Import required libraries
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np
import pandas as pd

def train_tabnet(train_df, val_df, feature_cols, response_col='ClaimYN', epochs=50):
    # Convert boolean columns to integers
    train_df = train_df.applymap(lambda x: int(x) if isinstance(x, bool) else x)
    val_df = val_df.applymap(lambda x: int(x) if isinstance(x, bool) else x)

    # Ensure all data is numeric and convert to numpy arrays
    X_train = train_df[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).values
    y_train = train_df[response_col].apply(pd.to_numeric, errors='coerce').fillna(0).values
    X_val = val_df[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).values
    y_val = val_df[response_col].apply(pd.to_numeric, errors='coerce').fillna(0).values

    # Print shapes of the arrays
    print("Shapes of the datasets:")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")

    # Verify the types of the arrays
    print("Data types:")
    print(f"X_train type: {type(X_train)}, y_train type: {type(y_train)}")
    print(f"X_val type: {type(X_val)}, y_val type: {type(y_val)}")

    # Print sample data
    print("Sample data:")
    print(f"X_train sample: {X_train[:5]}")
    print(f"y_train sample: {y_train[:5]}")
    print(f"X_val sample: {X_val[:5]}")
    print(f"y_val sample: {y_val[:5]}")

    # Assert that the arrays contain only numeric data
    assert np.issubdtype(X_train.dtype, np.number), "X_train contains non-numeric data"
    assert np.issubdtype(y_train.dtype, np.number), "y_train contains non-numeric data"
    assert np.issubdtype(X_val.dtype, np.number), "X_val contains non-numeric data"
    assert np.issubdtype(y_val.dtype, np.number), "y_val contains non-numeric data"

    clf = TabNetClassifier()

    clf.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        eval_name=['train', 'val'],
        eval_metric=['accuracy'],
        max_epochs=10,
        patience=100,
        batch_size=1024,
        virtual_batch_size=128,
        num_workers=0,
        weights=1,
        drop_last=False
    )

    return clf

# Assuming train_df_level1, val_df_level1, feature_cols_level1, etc., are already defined
clf_level1 = train_tabnet(train_df_level1, val_df_level1, feature_cols_level1)
clf_level2 = train_tabnet(train_df_level2, val_df_level2, feature_cols_level2)
clf_level3 = train_tabnet(train_df_level3, val_df_level3, feature_cols_level3)


Shapes of the datasets:
X_train: (64000, 110), y_train: (64000,)
X_val: (16000, 110), y_val: (16000,)
Data types:
X_train type: <class 'numpy.ndarray'>, y_train type: <class 'numpy.ndarray'>
X_val type: <class 'numpy.ndarray'>, y_val type: <class 'numpy.ndarray'>
Sample data:
X_train sample: [[3.66000000e+02 6.40000000e+01 2.00000000e+00 6.95000000e+02
  1.24274200e+04 8.00000000e+00 4.82191781e-01 3.25681330e+03
  1.51102077e-01 1.67734049e-01 1.68922553e-01 1.75627304e-01
  1.71069433e-01 1.31185908e-01 3.43586750e-02 2.19352600e-03
  0.00000000e+00 0.00000000e+00 8.38064741e-01 1.61935259e-01
  2.23870517e-01 2.12878450e-02 6.31935259e+00 1.10000000e+01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 3.40000000e+01 2.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.10000000e+01
  1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  5.80000000e+01 1.70000000e+01 2.00000000e+00 0.00000000e+00
  0.00000000e+00 0.000000

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.6325  | train_accuracy: 0.78583 | val_accuracy: 0.78012 |  0:00:03s
epoch 1  | loss: 0.57592 | train_accuracy: 0.73548 | val_accuracy: 0.73225 |  0:00:09s
epoch 2  | loss: 0.56626 | train_accuracy: 0.6738  | val_accuracy: 0.67338 |  0:00:13s
epoch 3  | loss: 0.56547 | train_accuracy: 0.60909 | val_accuracy: 0.61262 |  0:00:17s
epoch 4  | loss: 0.55606 | train_accuracy: 0.67339 | val_accuracy: 0.673   |  0:00:22s
epoch 5  | loss: 0.54594 | train_accuracy: 0.65309 | val_accuracy: 0.6515  |  0:00:27s
epoch 6  | loss: 0.5377  | train_accuracy: 0.70662 | val_accuracy: 0.7035  |  0:00:32s
epoch 7  | loss: 0.52202 | train_accuracy: 0.65916 | val_accuracy: 0.65575 |  0:00:36s
epoch 8  | loss: 0.51118 | train_accuracy: 0.70066 | val_accuracy: 0.69644 |  0:00:40s
epoch 9  | loss: 0.50141 | train_accuracy: 0.69269 | val_accuracy: 0.68444 |  0:00:45s
Stop training because you reached max_epochs = 10 with best_epoch = 0 and best_val_accuracy = 0.78012


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Shapes of the datasets:
X_train: (64000, 110), y_train: (64000,)
X_val: (16000, 110), y_val: (16000,)
Data types:
X_train type: <class 'numpy.ndarray'>, y_train type: <class 'numpy.ndarray'>
X_val type: <class 'numpy.ndarray'>, y_val type: <class 'numpy.ndarray'>
Sample data:
X_train sample: [[1.00000000e+00 5.51724138e-01 1.81818182e-01 5.71129707e-01
  2.19058050e-01 1.01265823e-01 4.80769231e-01 6.88778598e-02
  1.51378842e-01 1.67734049e-01 1.68922553e-01 1.75996906e-01
  1.71306277e-01 1.38587015e-01 3.52012753e-02 4.81308685e-03
  0.00000000e+00 0.00000000e+00 8.38064741e-01 1.61935259e-01
  2.26579899e-01 2.14340683e-02 8.99891509e-01 1.77133655e-02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 5.47504026e-02 3.22061192e-03 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.38410046e-05
  1.25837448e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00
  6.89483007e-05 2.02090567e-05 2.37755587e-06 0.00000000e+00
  0.00000000e+00 0.000000

...

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Shapes of the datasets:
X_train: (64000, 111), y_train: (64000,)
X_val: (16000, 111), y_val: (16000,)
Data types:
X_train type: <class 'numpy.ndarray'>, y_train type: <class 'numpy.ndarray'>
X_val type: <class 'numpy.ndarray'>, y_val type: <class 'numpy.ndarray'>
Sample data:
X_train sample: [[1.00000000e+00 5.51724138e-01 1.81818182e-01 5.71129707e-01
  2.19058050e-01 1.01265823e-01 4.80769231e-01 6.88778598e-02
  1.51378842e-01 1.67734049e-01 1.68922553e-01 1.75996906e-01
  1.71306277e-01 1.38587015e-01 3.52012753e-02 4.81308685e-03
  0.00000000e+00 0.00000000e+00 8.38064741e-01 1.61935259e-01
  2.26579899e-01 2.14340683e-02 8.99891509e-01 1.77133655e-02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 5.47504026e-02 3.22061192e-03 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.38410046e-05
  1.25837448e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00
  6.89483007e-05 2.02090567e-05 2.37755587e-06 0.00000000e+00
  0.00000000e+00 0.000000

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.68455 | train_accuracy: 0.04211 | val_accuracy: 0.04488 |  0:00:04s
epoch 1  | loss: 0.59187 | train_accuracy: 0.0422  | val_accuracy: 0.04494 |  0:00:08s
epoch 2  | loss: 0.58674 | train_accuracy: 0.05048 | val_accuracy: 0.05219 |  0:00:13s
epoch 3  | loss: 0.58849 | train_accuracy: 0.05091 | val_accuracy: 0.0525  |  0:00:17s
epoch 4  | loss: 0.58329 | train_accuracy: 0.393   | val_accuracy: 0.397   |  0:00:21s
epoch 5  | loss: 0.57294 | train_accuracy: 0.51016 | val_accuracy: 0.51081 |  0:00:26s
epoch 6  | loss: 0.56885 | train_accuracy: 0.61344 | val_accuracy: 0.61469 |  0:00:30s
epoch 7  | loss: 0.54927 | train_accuracy: 0.60753 | val_accuracy: 0.611   |  0:00:34s
epoch 8  | loss: 0.54174 | train_accuracy: 0.68238 | val_accuracy: 0.68562 |  0:00:39s
epoch 9  | loss: 0.53716 | train_accuracy: 0.70473 | val_accuracy: 0.70706 |  0:00:43s
Stop training because you reached max_epochs = 10 with best_epoch = 9 and best_val_accuracy = 0.70706


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
import pickle

# Save the model
with open('clf_level1.pkl', 'wb') as file:
    pickle.dump(clf_level1, file)

# Load the model
with open('clf_level1.pkl', 'rb') as file:
    clf_level1 = pickle.load(file)


In [ ]:
import pickle
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def evaluate_model(clf, test_df, feature_cols, response_col='ClaimYN'):
    # Ensure all data is numeric
    X_test = test_df[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)
    y_test = test_df[response_col].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)

    # Calculate predictions
    y_pred = clf.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

    print(f'Accuracy: {accuracy}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'F1-Score: {f1}')
    print(f'MCC: {mcc}')
    print(f'AUC: {auc}')

    # Plot loss and accuracy if available
    if hasattr(clf, 'history_'):
        history = clf.history_

        plt.figure(figsize=(12, 5))

        plt.subplot(1, 2, 1)
        if 'loss' in history:
            plt.plot(history['loss'], label='train_loss')
        if 'val_loss' in history:
            plt.plot(history['val_loss'], label='val_loss')
        plt.legend()
        plt.title('Loss')

        plt.subplot(1, 2, 2)
        if 'accuracy' in history:
            plt.plot(history['accuracy'], label='train_accuracy')
        if 'val_accuracy' in history:
            plt.plot(history['val_accuracy'], label='val_accuracy')
        plt.legend()
        plt.title('Accuracy')

        plt.show()
    else:
        print("No training history available for this model.")

def save_model(clf, filename):
    with open(filename, 'wb') as file:
        pickle.dump(clf, file)

def load_model(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

# Save the models
save_model(clf_level1, 'clf_level1.pkl')
save_model(clf_level2, 'clf_level2.pkl')
save_model(clf_level3, 'clf_level3.pkl')

# Load and evaluate the models
print("Evaluation for Level 1:")
clf_level1_loaded = load_model('clf_level1.pkl')
evaluate_model(clf_level1_loaded, test_df_level1, feature_cols_level1)

print("Evaluation for Level 2:")
clf_level2_loaded = load_model('clf_level2.pkl')
evaluate_model(clf_level2_loaded, test_df_level2, feature_cols_level2)

print("Evaluation for Level 3:")
clf_level3_loaded = load_model('clf_level3.pkl')
evaluate_model(clf_level3_loaded, test_df_level3, feature_cols_level3)


Evaluation for Level 1:
Accuracy: 0.7876
Recall: 0.5160919540229885
Precision: 0.1050046772684752
F1-Score: 0.17450446949086668
MCC: 0.1572383069760196
AUC: 0.7487822881554519
No training history available for this model.
Evaluation for Level 2:
Accuracy: 0.6396
Recall: 0.7770114942528735
Precision: 0.08790637191157347
F1-Score: 0.15794392523364484
MCC: 0.17206491317898748
AUC: 0.7735803726469227
No training history available for this model.
Evaluation for Level 3:
Accuracy: 0.7061
Recall: 0.7160919540229885
Precision: 0.0996162456028142
F1-Score: 0.17490174059517125
MCC: 0.18556337394284583
AUC: 0.7767039493844295
No training history available for this model.


In [ ]:
# Import required libraries for evaluation
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def evaluate_model(clf, test_df, feature_cols, response_col='ClaimYN'):
    # Ensure all data is numeric
    X_test = test_df[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)
    y_test = test_df[response_col].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)

    # Calculate predictions
    y_pred = clf.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

    print(f'Accuracy: {accuracy}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'F1-Score: {f1}')
    print(f'MCC: {mcc}')
    print(f'AUC: {auc}')

    # Plot loss and accuracy if available
    if hasattr(clf, 'history_'):
        history = clf.history_

        plt.figure(figsize=(12, 5))

        plt.subplot(1, 2, 1)
        if 'loss' in history:
            plt.plot(history['loss'], label='train_loss')
        if 'val_loss' in history:
            plt.plot(history['val_loss'], label='val_loss')
        plt.legend()
        plt.title('Loss')

        plt.subplot(1, 2, 2)
        if 'accuracy' in history:
            plt.plot(history['accuracy'], label='train_accuracy')
        if 'val_accuracy' in history:
            plt.plot(history['val_accuracy'], label='val_accuracy')
        plt.legend()
        plt.title('Accuracy')

        plt.show()
    else:
        print("No training history available for this model.")

print("Evaluation for Level 1:")
evaluate_model(clf_level1, test_df_level1, feature_cols_level1)

print("Evaluation for Level 2:")
evaluate_model(clf_level2, test_df_level2, feature_cols_level2)

print("Evaluation for Level 3:")
evaluate_model(clf_level3, test_df_level3, feature_cols_level3)


Evaluation for Level 1:
Accuracy: 0.8547
Recall: 0.5333333333333333
Precision: 0.15654520917678813
F1-Score: 0.24204486176317164
MCC: 0.23116406883165821
AUC: 0.7831592371613461
No training history available for this model.
Evaluation for Level 2:
Accuracy: 0.8496
Recall: 0.5264367816091954
Precision: 0.14996725605762934
F1-Score: 0.2334352701325178
MCC: 0.22157964434883623
AUC: 0.7805994376047731
No training history available for this model.
Evaluation for Level 3:
Accuracy: 0.84325
Recall: 0.5436781609195402
Precision: 0.14730613516038618
F1-Score: 0.23180592991913748
MCC: 0.22255822273699005
AUC: 0.7800172143410783
No training history available for this model.


In [ ]:
# Import required libraries for evaluation
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def evaluate_model(clf, test_df, feature_cols, response_col='ClaimYN'):
    # Ensure all data is numeric
    X_test = test_df[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)
    y_test = test_df[response_col].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)

    # Calculate predictions
    y_pred = clf.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

    print(f'Accuracy: {accuracy}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'F1-Score: {f1}')
    print(f'MCC: {mcc}')
    print(f'AUC: {auc}')

    # Plot loss and accuracy if available
    if hasattr(clf, 'history_'):
        history = clf.history_

        plt.figure(figsize=(12, 5))

        plt.subplot(1, 2, 1)
        if 'loss' in history:
            plt.plot(history['loss'], label='train_loss')
        if 'val_loss' in history:
            plt.plot(history['val_loss'], label='val_loss')
        plt.legend()
        plt.title('Loss')

        plt.subplot(1, 2, 2)
        if 'train_accuracy' in history:
            plt.plot(history['train_accuracy'], label='train_accuracy')
        if 'val_accuracy' in history:
            plt.plot(history['val_accuracy'], label='val_accuracy')
        plt.legend()
        plt.title('Accuracy')

        plt.show()
    else:
        print("No training history available for this model.")

print("Evaluation for Level 1:")
evaluate_model(clf_level1, test_df_level1, feature_cols_level1)

print("Evaluation for Level 2:")
evaluate_model(clf_level2, test_df_level2, feature_cols_level2)

print("Evaluation for Level 3:")
evaluate_model(clf_level3, test_df_level3, feature_cols_level3)


Evaluation for Level 1:
Accuracy: 0.8547
Recall: 0.5333333333333333
Precision: 0.15654520917678813
F1-Score: 0.24204486176317164
MCC: 0.23116406883165821
AUC: 0.7831592371613461
No training history available for this model.
Evaluation for Level 2:
Accuracy: 0.8496
Recall: 0.5264367816091954
Precision: 0.14996725605762934
F1-Score: 0.2334352701325178
MCC: 0.22157964434883623
AUC: 0.7805994376047731
No training history available for this model.
Evaluation for Level 3:
Accuracy: 0.84325
Recall: 0.5436781609195402
Precision: 0.14730613516038618
F1-Score: 0.23180592991913748
MCC: 0.22255822273699005
AUC: 0.7800172143410783
No training history available for this model.


In [ ]:
import pandas as pd
import pickle
from google.colab import drive

def evaluate_model(clf, test_df, feature_cols, response_col='ClaimYN', model_save_path=None, metrics_save_path=None):
    # Ensure all data is numeric
    X_test = test_df[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)
    y_test = test_df[response_col].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)

    # Calculate predictions
    y_pred = clf.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

    # Create a DataFrame for metrics
    metrics_df = pd.DataFrame({
        'Metric': ['Accuracy', 'Recall', 'Precision', 'F1-Score', 'MCC', 'AUC'],
        'Value': [accuracy, recall, precision, f1, mcc, auc]
    })

    # Save metrics to Google Drive
    if metrics_save_path:
        metrics_df.to_csv(metrics_save_path, index=False)
        print(f"Metrics saved to {metrics_save_path}")

    # Save the model to Google Drive
    if model_save_path:
        with open(model_save_path, 'wb') as model_file:
            pickle.dump(clf, model_file)
        print(f"Model saved to {model_save_path}")

    # Print metrics
    print(f'Accuracy: {accuracy}')
    print(f'Recall: {recall}')
    print(f'Precision: {precision}')
    print(f'F1-Score: {f1}')
    print(f'MCC: {mcc}')
    print(f'AUC: {auc}')

    # Plot loss and accuracy if available
    if hasattr(clf, 'history_'):
        history = clf.history_

        plt.figure(figsize=(12, 5))

        plt.subplot(1, 2, 1)
        if 'loss' in history:
            plt.plot(history['loss'], label='train_loss')
        if 'val_loss' in history:
            plt.plot(history['val_loss'], label='val_loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.title('Loss Over Epochs')

        plt.subplot(1, 2, 2)
        if 'train_accuracy' in history:
            plt.plot(history['train_accuracy'], label='train_accuracy')
        if 'val_accuracy' in history:
            plt.plot(history['val_accuracy'], label='val_accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.title('Accuracy Over Epochs')

        plt.tight_layout()
        plt.show()
    else:
        print("No training history available for this model.")

# Example usage
metrics_save_path_level1 = '/content/drive/My Drive/metrics_level1.csv'
model_save_path_level1 = '/content/drive/My Drive/model_level1.pkl'
evaluate_model(clf_level1, test_df_level1, feature_cols_level1, metrics_save_path=metrics_save_path_level1, model_save_path=model_save_path_level1)

metrics_save_path_level2 = '/content/drive/My Drive/metrics_level2.csv'
model_save_path_level2 = '/content/drive/My Drive/model_level2.pkl'
evaluate_model(clf_level2, test_df_level2, feature_cols_level2, metrics_save_path=metrics_save_path_level2, model_save_path=model_save_path_level2)

metrics_save_path_level3 = '/content/drive/My Drive/metrics_level3.csv'
model_save_path_level3 = '/content/drive/My Drive/model_level3.pkl'
evaluate_model(clf_level3, test_df_level3, feature_cols_level3, metrics_save_path=metrics_save_path_level3, model_save_path=model_save_path_level3)

Metrics saved to /content/drive/My Drive/metrics_level1.csv
Model saved to /content/drive/My Drive/model_level1.pkl
Accuracy: 0.7876
Recall: 0.5160919540229885
Precision: 0.1050046772684752
F1-Score: 0.17450446949086668
MCC: 0.1572383069760196
AUC: 0.7487822881554519
No training history available for this model.
Metrics saved to /content/drive/My Drive/metrics_level2.csv
Model saved to /content/drive/My Drive/model_level2.pkl
Accuracy: 0.7916
Recall: 0.6045977011494252
Precision: 0.12091954022988506
F1-Score: 0.20153256704980846
MCC: 0.20010176978151709
AUC: 0.7745766113284184
No training history available for this model.
Metrics saved to /content/drive/My Drive/metrics_level3.csv
Model saved to /content/drive/My Drive/model_level3.pkl
Accuracy: 0.798
Recall: 0.6011494252873564
Precision: 0.12405123339658444
F1-Score: 0.20566260322453794
MCC: 0.20409240530069214
AUC: 0.7800603853849344
No training history available for this model.
